In [1]:
import pandas as pd
from datetime import date, timedelta
import random
import xlsxwriter
import os
import random
random.seed(1234)
os.chdir("/gpfs/home/fatman01/PTRdatasetcompilation")

candidate_dataset = pd.read_csv("Candidatedatasetforsimulation.csv").drop_duplicates()
donor_dataset = pd.read_csv("Donordatasetforsimulation.csv").drop_duplicates()

candidate_dataset['date'] = pd.to_datetime(candidate_dataset['date'])
donor_dataset['DON_RECOV_DT'] = pd.to_datetime(donor_dataset['DON_RECOV_DT'])


In [2]:
candidate_dataset

,PX_ID,CenterId,JustId,events,date,t_start,t_stop,status_initial,status_criteria,status,...,cpo,api,papi,short_MCS_ever,CAN_GENDER,CAN_PERM_ZIP,CTR_TY,PRIMARY_ZIP,can_lat,can_long
0,1376474,382,44709.0,ChangeDt,2021-01-15,14,18,2.0,NaN,2.0,...,0.790265,0.612903,1.136364,0,M,27524.0,TX1,27710,36.0512,-78.8577
1,1376474,382,43917.0,"CAN_LISTING_DT,CANHX_BEGIN_DT,ChangeDt,HrSevFa...",2021-01-01,0,14,2.0,NaN,2.0,...,0.790265,0.612903,1.136364,0,M,27524.0,TX1,27710,36.0512,-78.8577
2,1376477,78,43922.0,"CAN_LISTING_DT,CANHX_BEGIN_DT,ChangeDt,HrSevFa...",2021-01-01,0,5,1.0,NaN,1.0,...,0.513604,0.787879,2.222222,1,M,91001.0,TX1,90095,33.7866,-118.2987
3,1376490,465,53223.0,ChangeDt,2021-06-27,174,362,6.0,NaN,6.0,...,0.762884,1.833333,1.500000,0,F,43230.0,TX1,44195,41.6857,-81.6728
4,1376490,465,62884.0,ChangeDt,2022-01-01,362,541,6.0,NaN,6.0,...,0.660012,3.500000,4.000000,0,F,43230.0,TX1,44195,41.6857,-81.6728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625,1386303,736,46903.0,"CAN_LISTING_DT,CANHX_BEGIN_DT,ChangeDt,HrSevFa...",2021-03-01,0,88,4.0,CriteriaLvadSupport,4.0,...,0.552064,1.176471,1.636364,0,M,99133.0,TX1,98195,47.6564,-122.3048
1626,1386303,736,51774.0,ChangeDt,2021-05-28,88,177,4.0,CriteriaLvadSupport,4.0,...,0.552064,1.176471,1.636364,0,M,99133.0,TX1,98195,47.6564,-122.3048
1627,1386314,656,46907.0,"CAN_LISTING_DT,CANHX_BEGIN_DT,ChangeDt,HrSevFa...",2021-03-01,0,14,2.0,CriteriaIabpSupport.stat2,2.0,...,0.800986,3.571429,3.250000,0,M,77043.0,TX1,77030,29.7041,-95.4010
1628,1386314,656,47642.0,"ChangeDt,HrSevFailSodiumDt,HrSevFailCreatinine...",2021-03-15,14,15,2.0,CriteriaIabpSupport.stat2,2.0,...,0.800986,3.571429,3.250000,0,M,77043.0,TX1,77030,29.7041,-95.4010


In [3]:
print(list(candidate_dataset.columns))

['PX_ID', 'CenterId', 'JustId', 'events', 'date', 't_start', 't_stop', 'status_initial', 'status_criteria', 'status', 'status1', 'ExtensionNumber', 'exception', 'listing_description', 'FormStatus_descrip', 'systolicBP', 'PCWP', 'central_venous_pressure', 'cardiac_output', 'cardiac_index', 'arterial_lactate', 'AST', 'creatinine', 'bilirubin', 'albumin', 'sodium', 'BNP', 'heart_rate', 'diastolicBP', 'PASP', 'PADP', 'hemoglobin', 'LDH', 'BUN', 'INR', 'CAN_ABO', 'CAN_ETHNICITY_SRTR', 'CAN_LISTING_CTR_CD', 'CAN_LISTING_CTR_TY', 'CAN_PRELIM_XMATCH_REQUEST', 'CAN_REM_CD', 'diagnosis', 'CAN_MAX_MILE', 'CAN_MIN_WGT', 'CAN_MAX_WGT', 'CAN_ACPT_GENDER', 'CAN_ACPT_DCD', 'CAN_ACPT_HIST_CAD', 'CAN_VENTILATOR', 'CAN_ACPT_HCV_POS', 'CAN_PRIMARY_PAY', 'CAN_HGT_CM', 'CAN_WGT_KG', 'CAN_BMI', 'CAN_ACPT_HBC_POS', 'CAN_EDUCATION', 'CAN_LIFE_SUPPORT', 'CAN_INHALED_NO', 'CAN_LIFE_SUPPORT_OTHER', 'CAN_WORK_INCOME', 'diabetes', 'dialysis', 'CAN_CEREB_VASC', 'CAN_IMPLANT_DEFIB', 'CAN_CARDIAC_SURG', 'CAN_RACE_WHIT

In [4]:
print(donor_dataset)

     PERS_ID DON_TY  DON_TX  DON_AGE_IN_MONTHS  DON_AGE DON_GENDER  \
0    8513691      C       6                429       35          M   
1    8513723      C       6                298       24          M   
2    8513740      C       6                517       43          M   
3    8513708      C       6                531       44          M   
4    8513777      C       6                200       16          M   
..       ...    ...     ...                ...      ...        ...   
437  8733369      C       6                282       23          F   
438  8733849      C       6                411       34          M   
439  8745029      C       6                558       46          F   
440  8741862      C       6                396       33          M   
441  8739223      C       6                575       47          M   

    DON_HOME_STATE  DON_RACE DON_RACE_SRTR DON_ETHNICITY_SRTR  ...  \
0               CA      1024         WHITE             LATINO  ...   
1               NV 

In [5]:
print(donor_dataset.columns)

Index(['PERS_ID', 'DON_TY', 'DON_TX', 'DON_AGE_IN_MONTHS', 'DON_AGE',
       'DON_GENDER', 'DON_HOME_STATE', 'DON_RACE', 'DON_RACE_SRTR',
       'DON_ETHNICITY_SRTR',
       ...
       'DON_SKIN_CANCER_TYPE', 'DON_STRONGYLOIDES',
       'DON_LEGALLY_DOCUMENT_DECISION', 'DON_WAS_AUTHORIZATION_OBTAINED',
       'DON_PREV_GASTRO_DISEASE', 'DON_OPO_CTR_ID', 'DONOR_ID', 'PRIMARY_ZIP',
       'don_lat', 'don_long'],
      dtype='object', length=262)


In [6]:
class CandidateWaitlist:
    def __init__(self):
        self.cumulative_candidates = []
        self.arrival_info = {}
        self.candidate_info_waiting = {}

        self.status_groups = {str(i): [] for i in range(1, 7)}
        self.status_groups['Inactive'] = []

        self.by_abo = {'O': [], 'A': [], 'B': [], 'AB': []}
        self.by_age = {'Adult': [], 'Pediatric': []}
        
        

    def add_candidate(self, px_id, listing_date, abo, age, status_initial, can_hgt_cm, can_cardiac_surg, can_listing_ctr_cd, can_lat, can_long):
        if px_id not in self.arrival_info:
            self.arrival_info[px_id] = {
                'listing_dates': [],
                'abo': abo,
                'age': age,
                'status_initial': status_initial,
                'can_hgt_cm': can_hgt_cm,
                'can_cardiac_surg': can_cardiac_surg,
                'can_listing_ctr_cd': can_listing_ctr_cd,
                'can_lat': can_lat,
                'can_long': can_long
            }
        self.arrival_info[px_id]['listing_dates'].append(listing_date)
        self.arrival_info[px_id]['listing_dates'].sort()
                
    

    def process_day(self, current_day, candidate_df):
        arrivals_today = 0
                            
        for px_id, info in self.arrival_info.items():
            if current_day in info['listing_dates'] and px_id not in self.cumulative_candidates:
                self.cumulative_candidates.append(px_id)
                arrivals_today += 1
                status = str(info['status_initial'])
                abo = info['abo'].upper()
                age = info['age']
                can_hgt_cm = info['can_hgt_cm']
                can_cardiac_surg = info['can_cardiac_surg'].upper()
                can_listing_ctr_cd = info['can_listing_ctr_cd']
                can_lat = info['can_lat']
                can_long = info['can_long']
                
                self.candidate_info_waiting[px_id] = {
                    'status': status,
                    'abo': abo,
                    'age': age,
                    'can_hgt_cm': can_hgt_cm,
                    'can_cardiac_surg': can_cardiac_surg,
                    'can_listing_ctr_cd': can_listing_ctr_cd,
                    'can_lat': can_lat,
                    'can_long': can_long,
                    'arrival_date': current_day,
                    'waiting_days': 0,
                    'waiting_days_by_status': {str(i): 0 for i in range(1, 7)},
                    'status_history': [(current_day, status)]   # track history
                }
                self.candidate_info_waiting[px_id]['waiting_days_by_status']['Inactive'] = 0
                self.status_groups[status].append(px_id) if status in self.status_groups else self.status_groups['Inactive'].append(px_id)
                self.by_abo[abo].append(px_id) if abo in self.by_abo else self.by_abo['AB'].append(px_id)
                self.by_age['Adult' if age >= 18 else 'Pediatric'].append(px_id)
                
                         
    
        for px_id, info in self.candidate_info_waiting.items():
            info['waiting_days'] = (current_day - info['arrival_date']).days

            status = info['status']
            if status in info['waiting_days_by_status']:
                info['waiting_days_by_status'][status] += 1
            else:
                info['waiting_days_by_status']['Inactive'] += 1

            df_today = candidate_df[(candidate_df['PX_ID'] == px_id) & (candidate_df['date'] == current_day)]
            if not df_today.empty:
                new_status = str(df_today.iloc[0]['status1'])
                if info['status'] != new_status:
                    old_status = info['status']
                    info['status'] = new_status
                    
                    if old_status in self.status_groups and px_id in self.status_groups[old_status]:
                        self.status_groups[old_status].remove(px_id)
                    self.status_groups[new_status].append(px_id) if new_status in self.status_groups else self.status_groups['Inactive'].append(px_id)
                    
                    info['status_history'].append((current_day, new_status))   # record change

        return arrivals_today

In [7]:
candidate_waitlist = CandidateWaitlist()

In [8]:
for _, row in candidate_dataset.iterrows():
    px_id = row['PX_ID']
    events = row['events'].split(',')
    
    if 'CAN_LISTING_DT' in events:
        listing_date = row['date']
        abo = row['CAN_ABO']
        age = round(row['age'])
        status_initial = row['status1']
        can_hgt_cm = row['CAN_HGT_CM']
        can_cardiac_surg = row['CAN_CARDIAC_SURG']
        can_listing_ctr_cd = row['CAN_LISTING_CTR_CD']
        can_lat = row['can_lat']
        can_long = row['can_long']
        
        candidate_waitlist.add_candidate(px_id, listing_date, abo, age, status_initial, can_hgt_cm, can_cardiac_surg, can_listing_ctr_cd,
                                        can_lat, can_long)


In [9]:
class DonorWaitlist:
    def __init__(self):
        self.cumulative_donors = []
        self.arrival_info = {}  
        self.donor_info_waiting = {} 

        self.by_abo = {'O': [], 'A': [], 'B': [], 'AB': []}
        self.by_age = {'Adult': [], 'Pediatric': []}


    def add_donor(self, donor_id, recovery_date, abo, age,  don_lat, don_long, don_hist_diab=None
                 ):
        if donor_id not in self.arrival_info:
            self.arrival_info[donor_id] = {
                'recovery_dates': [],
                'abo_don': abo,
                'age_don': age,
                'don_hist_diab': don_hist_diab,
                'don_lat': don_lat,
                'don_long': don_long
                
            }
        self.arrival_info[donor_id]['recovery_dates'].append(recovery_date)
        self.arrival_info[donor_id]['recovery_dates'].sort()



    
    def process_day(self, current_day):
        for donor_id, info in self.arrival_info.items():
            if current_day in info['recovery_dates'] and donor_id not in self.cumulative_donors:
                self.cumulative_donors.append(donor_id)
                abo = info['abo_don'].upper()
                age = info['age_don']
                don_lat = info['don_lat']
                don_long = info['don_long']
                don_hist_diab = info['don_hist_diab']  # <- include here

                
                self.donor_info_waiting[donor_id] = {
                    'abo': abo,
                    'age': age,
                    'arrival_date': current_day,
                    'don_lat': don_lat,
                    'don_long': don_long,
                    'don_hist_diab': don_hist_diab  # <- add diabetes info
                
                }
                if abo in self.by_abo:
                    self.by_abo[abo].append(donor_id)
                else:
                    self.by_abo['AB'].append(donor_id)
                if age >= 18:
                    self.by_age['Adult'].append(donor_id)
                else:
                    self.by_age['Pediatric'].append(donor_id)


In [10]:
donor_waitlist = DonorWaitlist()


In [11]:
for _, row in donor_dataset.iterrows():
    donor_id = row['DONOR_ID']
    recovery_date = row['DON_RECOV_DT']
    abo = row['DON_ABO']
    age = round(row['DON_AGE'])
    don_hist_diab = row['DON_HIST_DIAB']
    don_lat = row['don_lat']
    don_long = row['don_long']

    donor_waitlist.add_donor(donor_id, recovery_date, abo, age, don_lat, don_long, don_hist_diab
                            )


In [12]:
def normalize_bt(bt):
    bt = bt.strip().upper()
    if bt in ["A1", "A2"]:
        return "A"
    elif bt in ["A1B", "A2B"]:
        return "AB"
    return bt


donor_dataset['abo_normalized'] = donor_dataset['DON_ABO'].apply(normalize_bt)


In [13]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c * 0.621371  # convert km → miles


In [14]:
import pandas as pd
import random
from datetime import timedelta

df = pd.DataFrame(columns=["DONOR_ID", "PX_ID", "STATUS", "OFFER_DATE", "PTR_DISTANCE", "CLASSIFICATION", "WAITING DAYS",
                        "DONOR_AGE",  "DONOR_ABO",  "CAND_AGE", "CAND_ABO", "CAN_HGT_CM", "CAN_CARDIAC_SURG", "CAN_LISTING_CTR_CD",
                           "can_lat", "can_long", "don_lat", "don_long",
                           
                        "DON_HIST_DIAB"
                          ])


row_counter = 0
offered_pairs = set()


BLOOD_MATCH_RULES = {
    "O": (["O", "B"], ["A", "AB"]),
    "A": (["A", "AB"], []),
    "B": (["B", "AB"], []),
    "AB": (["AB"], []),
    "A1": (["AB"], []),
    "A2": (["AB"], [])
}

CLASSIFICATIONS = {
    "STATUS1_PRIMARY_500": "Classification 1",
    "STATUS1_SECONDARY_500": "Classification 2",
    "STATUS2_PRIMARY_500": "Classification 3",
    "STATUS2_SECONDARY_500": "Classification 4",
    "STATUS3_PRIMARY_250": "Classification 5",
    "STATUS3_SECONDARY_250": "Classification 6",
    "STATUS1_PRIMARY_1000": "Classification 7",
    "STATUS1_SECONDARY_1000": "Classification 8",
    "STATUS2_PRIMARY_1000": "Classification 9",
    "STATUS2_SECONDARY_1000": "Classification 10",
    "STATUS4_PRIMARY_250": "Classification 11",
    "STATUS4_SECONDARY_250": "Classification 12",
    "STATUS3_PRIMARY_500": "Classification 13",
    "STATUS3_SECONDARY_500": "Classification 14",
    "STATUS5_PRIMARY_250": "Classification 15",
    "STATUS5_SECONDARY_250": "Classification 16",
    "STATUS3_PRIMARY_1000": "Classification 17",
    "STATUS3_SECONDARY_1000": "Classification 18",
    "STATUS6_PRIMARY_250": "Classification 19",
    "STATUS6_SECONDARY_250": "Classification 20",
    "STATUS1_PRIMARY_1500": "Classification 21",
    "STATUS1_SECONDARY_1500": "Classification 22",
    "STATUS2_PRIMARY_1500": "Classification 23",
    "STATUS2_SECONDARY_1500": "Classification 24",
    "STATUS3_PRIMARY_1500": "Classification 25",
    "STATUS3_SECONDARY_1500": "Classification 26",
    "STATUS4_PRIMARY_500": "Classification 27",
    "STATUS4_SECONDARY_500": "Classification 28",
    "STATUS5_PRIMARY_500": "Classification 29",
    "STATUS5_SECONDARY_500": "Classification 30",
    "STATUS6_PRIMARY_500": "Classification 31",
    "STATUS6_SECONDARY_500": "Classification 32",
    "STATUS1_PRIMARY_1500": "Classification 33",
    "STATUS1_SECONDARY_2500": "Classification 34",
    "STATUS2_PRIMARY_2500": "Classification 35",
    "STATUS2_SECONDARY_2500": "Classification 36",
    "STATUS3_PRIMARY_2500": "Classification 37",
    "STATUS3_SECONDARY_2500": "Classification 38",
    "STATUS4_PRIMARY_1000": "Classification 39",
    "STATUS4_SECONDARY_1000": "Classification 40",
    "STATUS5_PRIMARY_1000": "Classification 41",
    "STATUS5_SECONDARY_1000": "Classification 42",
    "STATUS6_PRIMARY_1000": "Classification 43",
    "STATUS6_SECONDARY_1000": "Classification 44",
    "STATUS1_PRIMARY_Nation": "Classification 45",
    "STATUS1_SECONDARY_Nation": "Classification 46",
    "STATUS2_PRIMARY_Nation": "Classification 47",
    "STATUS2_SECONDARY_Nation": "Classification 48",
    "STATUS3_PRIMARY_Nation": "Classification 49",
    "STATUS3_SECONDARY_Nation": "Classification 50",
    "STATUS4_PRIMARY_1500": "Classification 51",
    "STATUS4_SECONDARY_1500": "Classification 52",
    "STATUS5_PRIMARY_1500": "Classification 53",
    "STATUS5_SECONDARY_1500": "Classification 54",
    "STATUS6_PRIMARY_1500": "Classification 55",
    "STATUS6_SECONDARY_1500": "Classification 56",
    "STATUS4_PRIMARY_2500": "Classification 57",
    "STATUS4_SECONDARY_2500": "Classification 58",
    "STATUS5_PRIMARY_2500": "Classification 59",
    "STATUS5_SECONDARY_2500": "Classification 60",
    "STATUS6_PRIMARY_2500": "Classification 61",
    "STATUS6_SECONDARY_2500": "Classification 62",
    "STATUS4_PRIMARY_Nation": "Classification 63",
    "STATUS4_SECONDARY_Nation": "Classification 64",
    "STATUS5_PRIMARY_Nation": "Classification 65",
    "STATUS5_SECONDARY_Nation": "Classification 66",
    "STATUS6_PRIMARY_Nation": "Classification 67",
    "STATUS6_SECONDARY_Nation": "Classification 68"
    
}



CLASSIFICATIONS1 = {
    
    "PED_STATUS1_PRIMARY_250": "Classification 3",
    "PED_STATUS1_SECONDARY_250": "Classification 4",
    "PED_STATUS2_PRIMARY_250": "Classification 5",
    "PED_STATUS2_SECONDARY_250": "Classification 6",
    "PED_STATUS1_PRIMARY_500": "Classification 9",
    "PED_STATUS1_SECONDARY_500": "Classification 10",
    "PED_STATUS2_PRIMARY_500": "Classification 11",
    "PED_STATUS2_SECONDARY_500": "Classification 12",
    "PED_STATUS3_PRIMARY_250": "Classification 13",
    "PED_STATUS3_SECONDARY_250": "Classification 14",
    "PED_STATUS4_PRIMARY_250": "Classification 15",
    "PED_STATUS4_SECONDARY_250": "Classification 16",
    "PED_STATUS5_PRIMARY_250": "Classification 17",
    "PED_STATUS5_SECONDARY_250": "Classification 18",
    "PED_STATUS3_PRIMARY_500": "Classification 19",
    "PED_STATUS3_SECONDARY_500": "Classification 20",
    "PED_STATUS4_PRIMARY_500": "Classification 21",
    "PED_STATUS4_SECONDARY_500": "Classification 22",
    "PED_STATUS5_PRIMARY_500": "Classification 23",
    "PED_STATUS5_SECONDARY_500": "Classification 24",
    "PED_STATUS6_PRIMARY_250": "Classification 27",
    "PED_STATUS6_SECONDARY_250": "Classification 28",
    "PED_STATUS1_PRIMARY_1000": "Classification 31",
    "PED_STATUS1_SECONDARY_1000": "Classification 32",
    "PED_STATUS2_PRIMARY_1000": "Classification 33",
    "PED_STATUS2_SECONDARY_1000": "Classification 34",
    "PED_STATUS3_PRIMARY_1000": "Classification 37",
    "PED_STATUS3_SECONDARY_1000": "Classification 38",
    "PED_STATUS4_PRIMARY_1000": "Classification 39",
    "PED_STATUS4_SECONDARY_1000": "Classification 40",
    "PED_STATUS5_PRIMARY_1000": "Classification 41",
    "PED_STATUS5_SECONDARY_1000": "Classification 42",
    "PED_STATUS6_PRIMARY_500": "Classification 45",
    "PED_STATUS6_SECONDARY_500": "Classification 46",
    "PED_STATUS6_PRIMARY_1000": "Classification 49",
    "PED_STATUS6_SECONDARY_1000": "Classification 50",
    "PED_STATUS1_PRIMARY_1500": "Classification 53",
    "PED_STATUS1_SECONDARY_1500": "Classification 54",
    "PED_STATUS2_PRIMARY_1500": "Classification 55",
    "PED_STATUS2_SECONDARY_1500": "Classification 56",
    "PED_STATUS3_PRIMARY_1500": "Classification 59",
    "PED_STATUS3_SECONDARY_1500": "Classification 60",
    "PED_STATUS4_PRIMARY_1500": "Classification 61",
    "PED_STATUS4_SECONDARY_1500": "Classification 62",
    "PED_STATUS5_PRIMARY_1500": "Classification 63",
    "PED_STATUS5_SECONDARY_1500": "Classification 64",
    "PED_STATUS6_PRIMARY_1500": "Classification 67",
    "PED_STATUS6_SECONDARY_1500": "Classification 68",
    
    "PED_STATUS1_PRIMARY_2500": "Classification 71",
    "PED_STATUS1_SECONDARY_2500": "Classification 72",
    "PED_STATUS2_PRIMARY_2500": "Classification 73",
    "PED_STATUS2_SECONDARY_2500": "Classification 74",
    "PED_STATUS3_PRIMARY_2500": "Classification 77",
    "PED_STATUS3_SECONDARY_2500": "Classification 78",
    "PED_STATUS4_PRIMARY_2500": "Classification 79",
    "PED_STATUS4_SECONDARY_2500": "Classification 80",
    "PED_STATUS5_PRIMARY_2500": "Classification 81",
    "PED_STATUS5_SECONDARY_2500": "Classification 82",
    "PED_STATUS6_PRIMARY_2500": "Classification 85",
    "PED_STATUS6_SECONDARY_2500": "Classification 86",

    "PED_STATUS1_PRIMARY_Nation": "Classification 89",
    "PED_STATUS1_SECONDARY_Nation": "Classification 90",
    "PED_STATUS2_PRIMARY_Nation": "Classification 91",
    "PED_STATUS2_SECONDARY_Nation": "Classification 92",
    "PED_STATUS3_PRIMARY_Nation": "Classification 95",
    "PED_STATUS3_SECONDARY_Nation": "Classification 96",
    "PED_STATUS4_PRIMARY_Nation": "Classification 97",
    "PED_STATUS4_SECONDARY_Nation": "Classification 98",
    "PED_STATUS5_PRIMARY_Nation": "Classification 99",
    "PED_STATUS5_SECONDARY_Nation": "Classification 100",
    "PED_STATUS6_PRIMARY_Nation": "Classification 103",
    "PED_STATUS6_SECONDARY_Nation": "Classification 104",
}
                         
                        

simulation_run_time = 20
start_day = min(candidate_dataset['date'].min(), donor_dataset['DON_RECOV_DT'].min())



for days_increment in range(simulation_run_time):
    current_day = start_day + timedelta(days=days_increment)

    candidate_waitlist.process_day(current_day, candidate_dataset)
    donor_waitlist.process_day(current_day)

    distance_matrix = []

    for DONOR_ID in donor_waitlist.cumulative_donors:
        # get donor coordinates
        donor_row = donor_dataset.loc[donor_dataset["DONOR_ID"] == DONOR_ID].iloc[0]
        don_lat = donor_row["don_lat"]
        don_lon = donor_row["don_long"]

        for PX_ID in candidate_waitlist.cumulative_candidates:
            # get candidate coordinates
            cand_row = candidate_dataset.loc[candidate_dataset["PX_ID"] == PX_ID].iloc[0]
            cand_lat = cand_row["can_lat"]
            cand_lon = cand_row["can_long"]

            # compute real Haversine distance
            PTR_DISTANCE = haversine(can_lat, can_long, don_lat, don_long)

            distance_matrix.append({
                "DONOR_ID": DONOR_ID,
                "PX_ID": PX_ID,
                "PTR_DISTANCE": PTR_DISTANCE
            })


            
    status_priority = {"1": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "Inactive": 999}
    for DONOR_ID in donor_waitlist.cumulative_donors:
        donor = donor_waitlist.donor_info_waiting[DONOR_ID]
        donor_age = donor["age"]
        donor_abo = donor["abo"]
        don_lat = donor["don_lat"]
        don_long = donor["don_long"]
        don_hist_diab = donor["don_hist_diab"]
    
    
        primary, secondary = BLOOD_MATCH_RULES[donor_abo]
        
        sorted_candidates = sorted(
        candidate_waitlist.cumulative_candidates,
        key=lambda px: status_priority.get(candidate_waitlist.candidate_info_waiting[px]["status"], 999))

            
        for PX_ID in sorted_candidates:
            cand = candidate_waitlist.candidate_info_waiting[PX_ID]
            cand_status = cand["status"]
            cand_abo = cand["abo"]
            cand_age = cand["age"]
            can_hgt_cm = cand["can_hgt_cm"]
            can_cardiac_surg = cand["can_cardiac_surg"]
            can_listing_ctr_cd = cand["can_listing_ctr_cd"]
            can_lat = cand["can_lat"]
            can_long = cand["can_long"]


            ptr_distance = next(
                (rec["PTR_DISTANCE"] for rec in distance_matrix if rec["DONOR_ID"] == DONOR_ID and rec["PX_ID"] == PX_ID),
                None
            )

            pair = (DONOR_ID, PX_ID)
            if pair in offered_pairs:
                continue

            classification = None
            
            if donor_age >= 18:
            
                if cand_status == "1":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS1_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS1_SECONDARY_500"]

                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS2_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS2_SECONDARY_500"]

                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS["STATUS3_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS["STATUS3_SECONDARY_250"]


                elif cand_status == "1":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS1_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS1_SECONDARY_1000"]


                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS2_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS2_SECONDARY_1000"]


                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS["STATUS4_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS["STATUS4_SECONDARY_250"]


                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS3_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS3_SECONDARY_500"]


                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS["STATUS5_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS["STATUS5_SECONDARY_250"]


                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS3_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS["STATUS3_SECONDARY_1000"]


                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS["STATUS6_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS["STATUS6_SECONDARY_250"]

                elif cand_status == "1":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS1_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS1_SECONDARY_1500"]

                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS2_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS2_SECONDARY_1500"] 

                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS3_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS3_SECONDARY_1500"] 

                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS4_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS4_SECONDARY_500"]


                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS5_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS5_SECONDARY_500"]

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS6_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS["STATUS6_SECONDARY_500"]   

                elif cand_status == "1":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS1_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS1_SECONDARY_2500"]

                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS2_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS2_SECONDARY_2500"] 

                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS3_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS3_SECONDARY_2500"] 


                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS4_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS4_SECONDARY_1000"]


                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS5_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS5_SECONDARY_1000"]

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS6_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS["STATUS6_SECONDARY_1000"]

                elif cand_status == "1":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS1_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS1_SECONDARY_Nation"]

                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS2_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS2_SECONDARY_Nation"] 

                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS3_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS3_SECONDARY_Nation"] 


                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS4_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS4_SECONDARY_1500"]


                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS5_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS5_SECONDARY_1500"]

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS6_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS["STATUS6_SECONDARY_1500"]


                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS4_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS4_SECONDARY_2500"]


                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS5_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS5_SECONDARY_2500"]

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS6_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS["STATUS6_SECONDARY_2500"]


                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS4_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS4_SECONDARY_Nation"]

                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS5_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS5_SECONDARY_Nation"] 

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS6_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS["STATUS6_SECONDARY_Nation"] 
            
            else:
                
            
                if cand_status == "1":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS1_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS1_SECONDARY_250"]

                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS2_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS2_SECONDARY_250"]

                elif cand_status == "1":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS1_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS1_SECONDARY_500"]


                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS2_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS2_SECONDARY_500"]


                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS3_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS3_SECONDARY_250"]


                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS4_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS4_SECONDARY_250"]


                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS5_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS5_SECONDARY_250"]

                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["STATUS3_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["STATUS3_SECONDARY_500"]   

                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS4_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS4_SECONDARY_500"]

                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS5_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS5_SECONDARY_500"] 
                        
                        
                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS5_PRIMARY_250"]
                    elif cand_abo in secondary and ptr_distance <= 250:
                        classification = CLASSIFICATIONS1["PED_STATUS5_SECONDARY_250"] 
                        


                elif cand_status == "1":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS1_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS1_SECONDARY_1000"]

                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS2_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS2_SECONDARY_1000"]
 

                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS3_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS3_SECONDARY_1000"]
                        

                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS4_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS4_SECONDARY_1000"]
                        
                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS5_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS5_SECONDARY_1000"]

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS6_PRIMARY_500"]
                    elif cand_abo in secondary and ptr_distance <= 500:
                        classification = CLASSIFICATIONS1["PED_STATUS6_SECONDARY_500"]
                        

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS6_PRIMARY_1000"]
                    elif cand_abo in secondary and ptr_distance <= 1000:
                        classification = CLASSIFICATIONS1["PED_STATUS6_SECONDARY_1000"]
                        

                elif cand_status == "1":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS1_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS1_SECONDARY_1500"]

                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS2_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS2_SECONDARY_1500"]

                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS3_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS3_SECONDARY_1500"]


                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS4_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS4_SECONDARY_1500"]


                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS5_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS5_SECONDARY_1500"]

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS6_PRIMARY_1500"]
                    elif cand_abo in secondary and ptr_distance <= 1500:
                        classification = CLASSIFICATIONS1["PED_STATUS6_SECONDARY_1500"]


                elif cand_status == "1":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS1_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS1_SECONDARY_2500"]

                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS2_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS2_SECONDARY_2500"]

                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS3_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS3_SECONDARY_2500"]


                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS4_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS4_SECONDARY_2500"]


                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS5_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS5_SECONDARY_2500"]

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS6_PRIMARY_2500"]
                    elif cand_abo in secondary and ptr_distance <= 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS6_SECONDARY_2500"]
                        
                        
                elif cand_status == "1":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS1_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS1_SECONDARY_Nation"]

                elif cand_status == "2":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS2_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS2_SECONDARY_Nation"]

                elif cand_status == "3":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS3_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS3_SECONDARY_Nation"]


                elif cand_status == "4":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS4_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS4_SECONDARY_Nation"]


                elif cand_status == "5":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS5_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS5_SECONDARY_Nation"]

                elif cand_status == "6":
                    if cand_abo in primary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS6_PRIMARY_Nation"]
                    elif cand_abo in secondary and ptr_distance > 2500:
                        classification = CLASSIFICATIONS1["PED_STATUS6_SECONDARY_Nation"]
                        

            if classification:
                df.loc[row_counter] = [
                    DONOR_ID,
                    PX_ID,
                    cand_status,
                    current_day.date(),
                    ptr_distance,
                    classification,
                    cand['waiting_days'],
                    donor_age,
                    donor_abo,
                    cand_age,
                    cand_abo,
                    can_hgt_cm,
                    can_cardiac_surg,
                    can_listing_ctr_cd,
                    can_lat,
                    can_long,
                    don_lat,
                    don_long,

                    
                    don_hist_diab,

                ]

                
                row_counter += 1
                offered_pairs.add(pair)

                days_waited = cand['waiting_days_by_status'][cand_status]
                print(f"Day {days_increment+1}: Donor {DONOR_ID} (ABO {donor_abo}, age {donor_age}) "
                      f"→ Candidate {PX_ID} (ABO {cand_abo}, age {cand_age}) "
                      f"(Status {cand_status}, {classification}, waited {cand['waiting_days']} days, "
                      f"distance {ptr_distance})")
                
                
        

adult_allocations = df[df["DONOR_AGE"] >= 18]
pediatric_allocations = df[df["DONOR_AGE"] < 18]


output_file = "Heart allocation results November14.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    adult_allocations.to_excel(writer, sheet_name="Adult_Donors", index=False)
    pediatric_allocations.to_excel(writer, sheet_name="Pediatric_Donors", index=False)

print(f"Saved offers with classification to {output_file} (Adult_Donors + Pediatric_Donors sheets).")



Day 8: Donor 599124 (ABO O, age 25) → Candidate 1376477 (ABO A, age 24) (Status 1, Classification 2, waited 7 days, distance 457.37298169909457)
Day 8: Donor 599124 (ABO O, age 25) → Candidate 1377095 (ABO O, age 39) (Status 1, Classification 1, waited 1 days, distance 457.37298169909457)
Day 8: Donor 599124 (ABO O, age 25) → Candidate 1377438 (ABO B, age 64) (Status 1, Classification 1, waited 0 days, distance 457.37298169909457)
Day 8: Donor 599124 (ABO O, age 25) → Candidate 1376474 (ABO O, age 56) (Status 2, Classification 3, waited 7 days, distance 457.37298169909457)
Day 8: Donor 599124 (ABO O, age 25) → Candidate 1376615 (ABO O, age 34) (Status 2, Classification 3, waited 4 days, distance 457.37298169909457)
Day 8: Donor 599124 (ABO O, age 25) → Candidate 1376781 (ABO A, age 51) (Status 2, Classification 4, waited 3 days, distance 457.37298169909457)
Day 8: Donor 599124 (ABO O, age 25) → Candidate 1376824 (ABO O, age 32) (Status 2, Classification 3, waited 3 days, distance 457.3

In [15]:
df['CLASS_NUM'] = df['CLASSIFICATION'].str.extract('(\d+)').astype(int)

In [16]:
df

,DONOR_ID,PX_ID,STATUS,OFFER_DATE,PTR_DISTANCE,CLASSIFICATION,WAITING DAYS,DONOR_AGE,DONOR_ABO,CAND_AGE,CAND_ABO,CAN_HGT_CM,CAN_CARDIAC_SURG,CAN_LISTING_CTR_CD,can_lat,can_long,don_lat,don_long,DON_HIST_DIAB,CLASS_NUM
0,599124,1376477,1,2021-01-08,457.372982,Classification 2,7,25,O,24,A,177.80,N,CAUC,33.7866,-118.2987,37.7562,-121.9522,1,2
1,599124,1377095,1,2021-01-08,457.372982,Classification 1,1,25,O,39,O,195.00,Y,MNMC,44.0032,-92.4835,37.7562,-121.9522,1,1
2,599124,1377438,1,2021-01-08,457.372982,Classification 1,0,25,O,64,B,172.72,N,CASU,37.4236,-122.1619,37.7562,-121.9522,1,1
3,599124,1376474,2,2021-01-08,457.372982,Classification 3,7,25,O,56,O,187.96,N,NCDU,36.0512,-78.8577,37.7562,-121.9522,1,3
4,599124,1376615,2,2021-01-08,457.372982,Classification 3,4,25,O,34,O,178.99,Y,MNMC,44.0032,-92.4835,37.7562,-121.9522,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,600087,1378057,4,2021-01-19,133.492794,Classification 11,6,19,AB,62,AB,162.56,Y,NEUN,41.2435,-95.9629,39.7218,-104.8951,1,11
1147,600087,1378138,4,2021-01-19,133.492794,Classification 11,6,19,AB,58,AB,180.34,N,CAUC,33.7866,-118.2987,39.7218,-104.8951,1,11
1148,600087,1376495,6,2021-01-19,133.492794,Classification 19,15,19,AB,52,AB,187.96,N,PAAG,40.4406,-79.9959,39.7218,-104.8951,1,19
1149,600087,1376986,6,2021-01-19,133.492794,Classification 19,13,19,AB,72,AB,181.00,N,WIUW,43.0696,-89.4239,39.7218,-104.8951,1,19


In [17]:
df['SEQUENCE_NUMBER'] = df.groupby(['DONOR_ID', 'OFFER_DATE'])['CLASS_NUM'] \
                           .rank(method='dense', ascending=True).astype(int)


In [18]:
df

,DONOR_ID,PX_ID,STATUS,OFFER_DATE,PTR_DISTANCE,CLASSIFICATION,WAITING DAYS,DONOR_AGE,DONOR_ABO,CAND_AGE,...,CAN_HGT_CM,CAN_CARDIAC_SURG,CAN_LISTING_CTR_CD,can_lat,can_long,don_lat,don_long,DON_HIST_DIAB,CLASS_NUM,SEQUENCE_NUMBER
0,599124,1376477,1,2021-01-08,457.372982,Classification 2,7,25,O,24,...,177.80,N,CAUC,33.7866,-118.2987,37.7562,-121.9522,1,2,2
1,599124,1377095,1,2021-01-08,457.372982,Classification 1,1,25,O,39,...,195.00,Y,MNMC,44.0032,-92.4835,37.7562,-121.9522,1,1,1
2,599124,1377438,1,2021-01-08,457.372982,Classification 1,0,25,O,64,...,172.72,N,CASU,37.4236,-122.1619,37.7562,-121.9522,1,1,1
3,599124,1376474,2,2021-01-08,457.372982,Classification 3,7,25,O,56,...,187.96,N,NCDU,36.0512,-78.8577,37.7562,-121.9522,1,3,3
4,599124,1376615,2,2021-01-08,457.372982,Classification 3,4,25,O,34,...,178.99,Y,MNMC,44.0032,-92.4835,37.7562,-121.9522,1,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,600087,1378057,4,2021-01-19,133.492794,Classification 11,6,19,AB,62,...,162.56,Y,NEUN,41.2435,-95.9629,39.7218,-104.8951,1,11,2
1147,600087,1378138,4,2021-01-19,133.492794,Classification 11,6,19,AB,58,...,180.34,N,CAUC,33.7866,-118.2987,39.7218,-104.8951,1,11,2
1148,600087,1376495,6,2021-01-19,133.492794,Classification 19,15,19,AB,52,...,187.96,N,PAAG,40.4406,-79.9959,39.7218,-104.8951,1,19,3
1149,600087,1376986,6,2021-01-19,133.492794,Classification 19,13,19,AB,72,...,181.00,N,WIUW,43.0696,-89.4239,39.7218,-104.8951,1,19,3


In [19]:
output_csv = "heart_allocation_with_sequence.csv"
df.to_csv(output_csv, index=False)

print(f"Saved DataFrame with SEQUENCE_NUMBER to {output_csv}")

Saved DataFrame with SEQUENCE_NUMBER to heart_allocation_with_sequence.csv


In [20]:
df['OFFER_NUMBER'] = df.groupby(['DONOR_ID', 'OFFER_DATE'])['SEQUENCE_NUMBER'] \
                        .rank(method='first').astype(int)


In [21]:
df

,DONOR_ID,PX_ID,STATUS,OFFER_DATE,PTR_DISTANCE,CLASSIFICATION,WAITING DAYS,DONOR_AGE,DONOR_ABO,CAND_AGE,...,CAN_CARDIAC_SURG,CAN_LISTING_CTR_CD,can_lat,can_long,don_lat,don_long,DON_HIST_DIAB,CLASS_NUM,SEQUENCE_NUMBER,OFFER_NUMBER
0,599124,1376477,1,2021-01-08,457.372982,Classification 2,7,25,O,24,...,N,CAUC,33.7866,-118.2987,37.7562,-121.9522,1,2,2,3
1,599124,1377095,1,2021-01-08,457.372982,Classification 1,1,25,O,39,...,Y,MNMC,44.0032,-92.4835,37.7562,-121.9522,1,1,1,1
2,599124,1377438,1,2021-01-08,457.372982,Classification 1,0,25,O,64,...,N,CASU,37.4236,-122.1619,37.7562,-121.9522,1,1,1,2
3,599124,1376474,2,2021-01-08,457.372982,Classification 3,7,25,O,56,...,N,NCDU,36.0512,-78.8577,37.7562,-121.9522,1,3,3,4
4,599124,1376615,2,2021-01-08,457.372982,Classification 3,4,25,O,34,...,Y,MNMC,44.0032,-92.4835,37.7562,-121.9522,1,3,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,600087,1378057,4,2021-01-19,133.492794,Classification 11,6,19,AB,62,...,Y,NEUN,41.2435,-95.9629,39.7218,-104.8951,1,11,2,3
1147,600087,1378138,4,2021-01-19,133.492794,Classification 11,6,19,AB,58,...,N,CAUC,33.7866,-118.2987,39.7218,-104.8951,1,11,2,4
1148,600087,1376495,6,2021-01-19,133.492794,Classification 19,15,19,AB,52,...,N,PAAG,40.4406,-79.9959,39.7218,-104.8951,1,19,3,5
1149,600087,1376986,6,2021-01-19,133.492794,Classification 19,13,19,AB,72,...,N,WIUW,43.0696,-89.4239,39.7218,-104.8951,1,19,3,6


In [22]:
output_csv = "heart_allocation_with_sequence1.csv"
df.to_csv(output_csv, index=False)

print(f"Saved DataFrame with SEQUENCE_NUMBER to {output_csv}")

Saved DataFrame with SEQUENCE_NUMBER to heart_allocation_with_sequence1.csv


In [23]:
df['OFFER_NUMBER'] = df['OFFER_NUMBER'].astype(int)

def compute_center_number(group):
    # Sort by OFFER_NUMBER
    group = group.sort_values('OFFER_NUMBER')
    unique_centers = set()
    center_numbers = []

    for _, row in group.iterrows():
        unique_centers.add(row['CAN_LISTING_CTR_CD'])
        center_numbers.append(len(unique_centers))

    group['CENTER_NUMBER'] = center_numbers
    return group

df = df.groupby(['DONOR_ID', 'OFFER_DATE']).apply(compute_center_number).reset_index(drop=True)


In [24]:
df.to_csv("heart_allocation_with_center_number.csv", index=False)

In [25]:
# center rank per center
df['CENTER_RANK'] = df.groupby('CAN_LISTING_CTR_CD')['OFFER_NUMBER'] \
                      .rank(method='first').astype(int)

In [26]:
df.to_csv("heart_allocation_with_center_rank.csv", index=False)

In [27]:
df

,DONOR_ID,PX_ID,STATUS,OFFER_DATE,PTR_DISTANCE,CLASSIFICATION,WAITING DAYS,DONOR_AGE,DONOR_ABO,CAND_AGE,...,can_lat,can_long,don_lat,don_long,DON_HIST_DIAB,CLASS_NUM,SEQUENCE_NUMBER,OFFER_NUMBER,CENTER_NUMBER,CENTER_RANK
0,599124,1377095,1,2021-01-08,457.372982,Classification 1,1,25,O,39,...,44.0032,-92.4835,37.7562,-121.9522,1,1,1,1,1,1
1,599124,1377438,1,2021-01-08,457.372982,Classification 1,0,25,O,64,...,37.4236,-122.1619,37.7562,-121.9522,1,1,1,2,2,4
2,599124,1376477,1,2021-01-08,457.372982,Classification 2,7,25,O,24,...,33.7866,-118.2987,37.7562,-121.9522,1,2,2,3,3,7
3,599124,1376474,2,2021-01-08,457.372982,Classification 3,7,25,O,56,...,36.0512,-78.8577,37.7562,-121.9522,1,3,3,4,4,7
4,599124,1376615,2,2021-01-08,457.372982,Classification 3,4,25,O,34,...,44.0032,-92.4835,37.7562,-121.9522,1,3,3,5,4,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,600362,1377358,6,2021-01-19,248.169920,Classification 19,11,29,B,67,...,39.9487,-75.1593,43.0834,-77.6344,1,19,6,22,19,3
1147,600362,1377866,6,2021-01-19,248.169920,Classification 19,7,29,B,71,...,32.7948,-96.7697,43.0834,-77.6344,1,19,6,23,20,9
1148,600362,1378389,6,2021-01-19,248.169920,Classification 19,5,29,B,66,...,29.9614,-90.1577,43.0834,-77.6344,1,19,6,24,21,2
1149,600362,1378809,6,2021-01-19,248.169920,Classification 19,1,29,B,70,...,40.0118,-75.1456,43.0834,-77.6344,1,19,6,25,22,2


In [28]:
import numpy as np
import pandas as pd

# Create months_since_listing variable
df['months_since_listing'] = df['WAITING DAYS'] / 30

# Coefficients dictionary
coeffs = {
    'DON_AGE': -0.027990004,
    'CAND_AGE': 0.091829545,
    'months_since_listing': -0.164738511,
    'DON_ABO_O_x_CAN_ABO_AB': 0.2866962289435255,
    'DON_ABO_O_x_CAN_ABO_O': 0.21426098599116786,
    'DON_ABO_O_x_CAN_ABO_B': 0.11014061161460756,
    'DON_ABO_B_x_CAN_ABO_AB': -0.061633049,
    'DON_ABO_B': 0.10793821304721858,
    'CAND_ABO_B': 0.2797118733847424,
    'CAND_ABO_O': 0.21426098599116786,
    'CAND_ABO_AB': 0.004621379,

    'CAN_HGT_CM':	-0.048940148,

    'CAN_CARDIAC_SURG_N': 0.131376065,
    'CAN_CARDIAC_SURG_Y': -0.102414329,
    'CAN_CARDIAC_SURG_U': 0.015241231,

    
    'DON_HIST_DIAB_1': 0.0556,
    'DON_HIST_DIAB_2': -0.0282,
    'DON_HIST_DIAB_3': -0.020245,
    'DON_HIST_DIAB_4': 0.2059,
    'DON_HIST_DIAB_998': -0.1658,


    'CAN_LISTING_CTR_CD_PAAG': 3.86217985,
    'CAN_LISTING_CTR_CD_MIUM': 3.756580047,
    'CAN_LISTING_CTR_CD_PACH': 3.545231298,
    'CAN_LISTING_CTR_CD_PAPT':	3.527019823,
    'CAN_LISTING_CTR_CD_WVWU':	3.201054036,
    'CAN_LISTING_CTR_CD_MABI':	2.164735171,
    'CAN_LISTING_CTR_CD_INSV':	2.151496834,
    'CAN_LISTING_CTR_CD_MDUM':	2.045661168,
    'CAN_LISTING_CTR_CD_TNBM':	1.975990279,
    'CAN_LISTING_CTR_CD_ILCH':	-1.889446385,
    'CAN_LISTING_CTR_CD_OHCM':	-1.730331972,
    'CAN_LISTING_CTR_CD_MISH':	1.692320551,
    'CAN_LISTING_CTR_CD_MDJH':	1.679080189,
    'CAN_LISTING_CTR_CD_MIHF':	1.592222882,
    'CAN_LISTING_CTR_CD_OKBC':	-1.42845379,
    'CAN_LISTING_CTR_CD_VAUV':	-1.428090139,
    'CAN_LISTING_CTR_CD_WISL':	1.417856072,
    'CAN_LISTING_CTR_CD_OHUC':	-1.386100313,
    'CAN_LISTING_CTR_CD_FLUF':	-1.318046521,
    'CAN_LISTING_CTR_CD_LAOF':	-1.309369063,
    'CAN_LISTING_CTR_CD_CASH':	-1.26495334,
    'CAN_LISTING_CTR_CD_FLLM':	1.218561483,
    'CAN_LISTING_CTR_CD_ARBH':	-1.172026181,
    'CAN_LISTING_CTR_CD_MACH':	-1.155661924,
    'CAN_LISTING_CTR_CD_ILCM':	-1.155239828,
    'CAN_LISTING_CTR_CD_NYFL':	1.116878318,
    'CAN_LISTING_CTR_CD_NJBI':	-1.107620674,
    'CAN_LISTING_CTR_CD_PATJ':	-1.09917855,
    'CAN_LISTING_CTR_CD_PATU':	1.093579874,
    'CAN_LISTING_CTR_CD_NYMS':	-1.084643834,
    'CAN_LISTING_CTR_CD_FLFH':	1.082008205,
    'CAN_LISTING_CTR_CD_KYUK':	1.05078987,
    'CAN_LISTING_CTR_CD_CACL':	-1.038006354,
    'CAN_LISTING_CTR_CD_COCH':	-1.027583274,
    'CAN_LISTING_CTR_CD_FLJM':	1.025509401,
    'CAN_LISTING_CTR_CD_GAEM':	-1.024592499,
    'CAN_LISTING_CTR_CD_ILLU':	-1.022357295,
    'CAN_LISTING_CTR_CD_ILNM':	-0.985143886,
    'CAN_LISTING_CTR_CD_TXHS':	-0.947880894,
    'CAN_LISTING_CTR_CD_MOCH':	-0.946227585,
    'CAN_LISTING_CTR_CD_MOLH': 0.927596216,
    'CAN_LISTING_CTR_CD_NCMH':	-0.907448743,
    'CAN_LISTING_CTR_CD_NJRW':	-0.887569468,
    'CAN_LISTING_CTR_CD_DCWH':	-0.8782108,
    'CAN_LISTING_CTR_CD_CASU':	0.861580456,
    'CAN_LISTING_CTR_CD_TXHD':	-0.844272834,
    'CAN_LISTING_CTR_CD_MAMG':	0.831394219,
    'CAN_LISTING_CTR_CD_MAPB':	-0.81718989,
    'CAN_LISTING_CTR_CD_NYCP':	0.809104915,
    'CAN_LISTING_CTR_CD_UTLD':	-0.785361862,
    'CAN_LISTING_CTR_CD_TXMH':	-0.783113884,
    'CAN_LISTING_CTR_CD_UTMC':	-0.779202675,
    'CAN_LISTING_CTR_CD_GAPH':	0.765992301,
    'CAN_LISTING_CTR_CD_CAUH':	-0.74682938,
    'CAN_LISTING_CTR_CD_INIM':	-0.743456506,
    'CAN_LISTING_CTR_CD_IAIV':	-0.740978428,
    'CAN_LISTING_CTR_CD_MSUM':	-0.72101965,
    'CAN_LISTING_CTR_CD_NYNY':	-0.717722686,
    'CAN_LISTING_CTR_CD_TNST':	0.709217743,
    'CAN_LISTING_CTR_CD_TNVU':	0.706117853,
    'CAN_LISTING_CTR_CD_FLMR':	-0.702165166,
    'CAN_LISTING_CTR_CD_TXHH':	-0.691563822,
    'CAN_LISTING_CTR_CD_PRCC':	0.689337527,
    'CAN_LISTING_CTR_CD_FLCC':	0.664099516,
    'CAN_LISTING_CTR_CD_CAUC':	-0.663807058,
    'CAN_LISTING_CTR_CD_TXCM':	-0.645289388,
    'CAN_LISTING_CTR_CD_MNAN':	0.593271354,
    'CAN_LISTING_CTR_CD_ILSF':	-0.59149078,
    'CAN_LISTING_CTR_CD_ALUA':	-0.578169787,
    'CAN_LISTING_CTR_CD_TXCT':	-0.573656953,
    'CAN_LISTING_CTR_CD_MOCM':	-0.554393692,
    'CAN_LISTING_CTR_CD_ORSV':	-0.543866969,
    'CAN_LISTING_CTR_CD_FLSL':	-0.541420864,
    'CAN_LISTING_CTR_CD_WISE':	-0.538756177,
    'CAN_LISTING_CTR_CD_CASF':	0.531558427,
    'CAN_LISTING_CTR_CD_AZMC':	0.504883168,
    'CAN_LISTING_CTR_CD_WICH':	-0.499831923,
    'CAN_LISTING_CTR_CD_MICH':	-0.495367359,
    'CAN_LISTING_CTR_CD_ORUO':	0.488243716,
    'CAN_LISTING_CTR_CD_KYJH':	-0.479833862,
    'CAN_LISTING_CTR_CD_WASH':	-0.47717159,
    'CAN_LISTING_CTR_CD_PACP':	-0.463393057,
    'CAN_LISTING_CTR_CD_NCBG':	0.447899236,
    'CAN_LISTING_CTR_CD_PAUP':	0.446461044,
    'CAN_LISTING_CTR_CD_AZUA':	0.442651976,
    'CAN_LISTING_CTR_CD_CASD':	0.439600163,
    'CAN_LISTING_CTR_CD_CASG':	-0.437843399,
    'CAN_LISTING_CTR_CD_VAFH':	-0.431552081,
    'CAN_LISTING_CTR_CD_OHUH':	-0.425297588,
    'CAN_LISTING_CTR_CD_SCCH':	-0.416544076,
    'CAN_LISTING_CTR_CD_TXTC':	-0.410950581,
    'CAN_LISTING_CTR_CD_NEUN':	0.407532975,
    'CAN_LISTING_CTR_CD_KYKC':	-0.406504289,
    'CAN_LISTING_CTR_CD_TXJS':	-0.396204718,
    'CAN_LISTING_CTR_CD_TXVA':	-0.391525139,
    'CAN_LISTING_CTR_CD_MOBH':	-0.383170274,
    'CAN_LISTING_CTR_CD_WAUW':	0.359539863,
    'CAN_LISTING_CTR_CD_ARCH':	-0.356144201,
    'CAN_LISTING_CTR_CD_AZGS':	0.356032571,
    'CAN_LISTING_CTR_CD_VAMV':	-0.351423552,
    'CAN_LISTING_CTR_CD_CALL':	-0.348305353,
    'CAN_LISTING_CTR_CD_FLJD':	0.31336233,
    'CAN_LISTING_CTR_CD_TXDL':	-0.311097099,
    'CAN_LISTING_CTR_CD_FLAC':	-0.310964091,
    'CAN_LISTING_CTR_CD_TXSP':	-0.305996375,
    'CAN_LISTING_CTR_CD_NYUC':	-0.292485416,
    'CAN_LISTING_CTR_CD_INLH':	0.287268005,
    'CAN_LISTING_CTR_CD_VANG':	0.284703152,
    'CAN_LISTING_CTR_CD_OHCC':	0.284623184,
    'CAN_LISTING_CTR_CD_COUC':	-0.245875349,
    'CAN_LISTING_CTR_CD_MNMC':	0.244184435,
    'CAN_LISTING_CTR_CD_AZCH':	-0.240450349,
    'CAN_LISTING_CTR_CD_NCDU':	0.229962701,
    'CAN_LISTING_CTR_CD_SCMU':	-0.200598724,
    'CAN_LISTING_CTR_CD_OHTC':	0.194631015,
    'CAN_LISTING_CTR_CD_CACS':	-0.192784904,
    'CAN_LISTING_CTR_CD_NYNS':	-0.192158073,
    'CAN_LISTING_CTR_CD_PAHE':	-0.185186074,
    'CAN_LISTING_CTR_CD_TXTX':	0.180687431,
    'CAN_LISTING_CTR_CD_NCCM':	-0.17360857,
    'CAN_LISTING_CTR_CD_CAPM':	-0.172037295,
    'CAN_LISTING_CTR_CD_CAPC':	-0.170655824,
    'CAN_LISTING_CTR_CD_NYMA':	-0.167547046,
    'CAN_LISTING_CTR_CD_KSUK':	0.16102058,
    'CAN_LISTING_CTR_CD_ILUC':	-0.160764514,
    'CAN_LISTING_CTR_CD_TXHI':	-0.130214445,
    'CAN_LISTING_CTR_CD_NYWC':	0.122860924,
    'CAN_LISTING_CTR_CD_MNUM':	-0.121785281,
    'CAN_LISTING_CTR_CD_WACH':	-0.114052196,
    'CAN_LISTING_CTR_CD_CTYN':	-0.113463928,
    'CAN_LISTING_CTR_CD_FLTG':	0.09478992,
    'CAN_LISTING_CTR_CD_VAMC':	-0.070959495,
    'CAN_LISTING_CTR_CD_MOCG':	-0.058526548,
    'CAN_LISTING_CTR_CD_CTHH':	-0.056522586,
    'CAN_LISTING_CTR_CD_TXSW':	0.052120924,
    'CAN_LISTING_CTR_CD_OHCH':	0.050143013,
    'CAN_LISTING_CTR_CD_OHOU':	0.030391733,
    'CAN_LISTING_CTR_CD_MANM':	-0.028109519,
    'CAN_LISTING_CTR_CD_WIUW':	0.017401535,

    'CENTER_NUMBER':	-0.980360228,	
    'CENTER_RANK':	-0.608281525,	
    'OFFER_NUMBER':	0.185548241,
   'PTR_DISTANCE':	-0.649830082	
    


}

intercept = 0


def compute_acceptance_probability(donor_age, cand_age, months_since_listing,
                                   donor_abo, cand_abo, can_hgt_cm, can_cardiac_surg, can_listing_ctr_cd,
                                   CENTER_NUMBER, CENTER_RANK, OFFER_NUMBER, PTR_DISTANCE,
                                   
                                   don_hist_diab
                                  ):
    linear_sum = (
        coeffs['DON_AGE'] * donor_age +
        coeffs['CAND_AGE'] * cand_age +
        coeffs['CAN_HGT_CM'] * can_hgt_cm  +
        coeffs['months_since_listing'] * months_since_listing +
        coeffs['CENTER_NUMBER'] * CENTER_NUMBER +
        coeffs['CENTER_RANK'] * CENTER_RANK +
        coeffs['OFFER_NUMBER'] * OFFER_NUMBER +
        coeffs['PTR_DISTANCE'] * PTR_DISTANCE +
        
        intercept
    )

    # ABO interaction terms
    key_interaction = f'DON_ABO_{donor_abo}_x_CAND_ABO_{cand_abo}'
    linear_sum += coeffs.get(key_interaction, 0)

    # Main ABO terms
    linear_sum += coeffs.get(f'DON_ABO_{donor_abo}', 0)
    linear_sum += coeffs.get(f'CAN_ABO_{cand_abo}', 0)

    # Candidate cardiac surgery
    can_cardiac_surg_key = f'CAN_CARD_{can_cardiac_surg}'
    linear_sum += coeffs.get(can_cardiac_surg_key, 0)

    # Candidate listing center code
    can_listing_ctr_cd_key = f'CAN_LISTING_CTR_CD_{can_listing_ctr_cd}'
    linear_sum += coeffs.get(can_listing_ctr_cd_key, 0)
    
    # Donor diabetes history
    don_diab_key = f'DON_HIST_DIAB_{don_hist_diab}'
    linear_sum += coeffs.get(don_diab_key, 0)
    
    

    prob = 1 / (1 + np.exp(-linear_sum))
    return prob



adult_df_lr = df[df["DONOR_AGE"] >= 18].copy()
pediatric_df_lr = df[df["DONOR_AGE"] < 18].copy()


for subset_df in [adult_df_lr, pediatric_df_lr]:
    subset_df['accept_prob_lr'] = subset_df.apply(
        lambda row: compute_acceptance_probability(
            donor_age=row['DONOR_AGE'],
            cand_age=row['CAND_AGE'],
            can_hgt_cm = row['CAN_HGT_CM'],
            can_cardiac_surg = row['CAN_CARDIAC_SURG'],
            can_listing_ctr_cd = row['CAN_LISTING_CTR_CD'],
            months_since_listing=row['months_since_listing'],
            donor_abo=row['DONOR_ABO'],
            cand_abo=row['CAND_ABO'],
            CENTER_NUMBER = row['CENTER_NUMBER'],
            CENTER_RANK = row['CENTER_RANK'],
            OFFER_NUMBER = row['OFFER_NUMBER'],
            PTR_DISTANCE = row['PTR_DISTANCE'],
            don_hist_diab=row['DON_HIST_DIAB']              
        ),
        axis=1
    )

    np.random.seed(42)
    subset_df['accepted_LR'] = subset_df['accept_prob_lr'].apply(lambda p: np.random.rand() < p)


print(f"Adult accepted offers: {adult_df_lr['accepted_LR'].sum()} / {len(adult_df_lr)}")
print(f"Pediatric accepted offers: {pediatric_df_lr['accepted_LR'].sum()} / {len(pediatric_df_lr)}")


output_file = "Accepted_Heart_Offers_Adults_Pediatrics_Nov14_YES_NO.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    adult_df_lr.to_excel(writer, sheet_name="Adult_Donors", index=False)
    pediatric_df_lr.to_excel(writer, sheet_name="Pediatric_Donors", index=False)

print(f"Saved accepted offers to {output_file}.")


Adult accepted offers: 0 / 1103
Pediatric accepted offers: 0 / 48
Saved accepted offers to Accepted_Heart_Offers_Adults_Pediatrics_Nov14_YES_NO.xlsx.
